In [1]:
import polars as pl
import sys
import os
sys.path.append("/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Fantasy_Football")

In [2]:
print(pl.__version__)

1.6.0


In [ ]:
def extract_year(filename):
    """Extracts the year from a filename that ends with a pattern like "_YYYY.json".

    Args:
        filename (str): The filename to extract the year from.

    Returns:
        int: The extracted year, or None if the year could not be extracted.

    Raises:
        ValueError: If the filename does not contain a valid year component.

    Example:
        >>> extract_year("data_2023.json")
        2023
        >>> extract_year("file_without_year.txt")
        None
    """

    try:
        # Split the filename by underscores and take the last part
        last_part = filename.split("_")[-1]

        # Split the last part by dots and take the first part (assumed to be the year)
        year_str = last_part.split(".")[0]

        # Convert the extracted year string to an integer
        return int(year_str)

    except (ValueError, IndexError):
        # Handle cases where the filename doesn't have expected format
        return None



def extract_year(filename):
    return int(filename.split("_")[-1].split(".")[0])

In [ ]:
def concatenate_json_files(directory_path, chunksize=None):
    all_dfs = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)

            # Extract the year from the filename
            year = extract_year(filename)

            # Read the JSON file into a DataFrame
            if chunksize is None:
                df = pl.read_json(file_path)
            else:
                df = pl.read_json(file_path, chunksize=chunksize)

            # Add a new column with the extracted year
            df = df.with_columns(Season=pl.lit(year))

            all_dfs.append(df)

    return pl.concat(all_dfs)

In [ ]:
def write_delta(read_path,write_path,mode):
    df = concatenate_json_files(read_path)
    try:
        df.write_delta(
            write_path,
            mode=mode
        )
        print(f"delta table succesfully created in path:{write_path}")
    except Exception as e:
        print(e)

In [221]:
read_path = '/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Raw/Team_Stats/special-teams/scoring'
write_path = '/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Bronze/staging.team_stats/special-teams/scoring'
mode = 'overwrite'
write_delta(read_path,write_path,mode)

delta table succesfully created in path:/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Bronze/staging.team_stats/special-teams/scoring


In [3]:
def read_delta(path):
    try:
        df = pl.read_delta(path)
        print(f"delta table successfully read from path: {path}")
        return df
    except Exception as e:
        print(e)

In [222]:
def read_csv(path):
    try:
        df = pl.read_csv(path)
        print(f"csv succesfully read from path: {path}")
        return df
    except Exception as e:
        print(e)

In [225]:
path = '/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Audit/path_metadata.csv'
df = read_csv(path)
#display(df)

csv succesfully read from path: /Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Audit/path_metadata.csv


In [ ]:
path = "/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Audit/path_metadata"
df.write_delta(path,mode="overwrite")

In [ ]:
path = '/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Audit/path_metadata'
df = read_delta(path).filter(pl.col('category')=='player_stats')
display(df)

delta table successfully read from path: /Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Audit/path_metadata


id,category,sub_category,type,raw_path,bronze_path,silver_path,gold_path,created_by,created_date,modified_date
i64,str,str,str,str,str,str,str,str,str,str
1,"""player_stats""",null,"""field-goals""","""/Users/trevera…",null,null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""
2,"""player_stats""",null,"""fumbles""","""/Users/trevera…",null,null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""
3,"""player_stats""",null,"""interceptions""","""/Users/trevera…",null,null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""
4,"""player_stats""",null,"""kickoff-return…","""/Users/trevera…",null,null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""
5,"""player_stats""",null,"""kickoffs""","""/Users/trevera…",null,null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""
6,"""player_stats""",null,"""passing""","""/Users/trevera…","""/Users/trevera…",null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""
7,"""player_stats""",null,"""punt-returns""","""/Users/trevera…",null,null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""
8,"""player_stats""",null,"""punts""","""/Users/trevera…",null,null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""
9,"""player_stats""",null,"""receiving""","""/Users/trevera…","""/Users/trevera…",null,null,"""Trever A""","""4/1/24 23:26""","""4/1/24 23:26"""


In [1]:
path = '/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Bronze/receiving'
df_receiving = read_delta(path)
display(df_receiving)

NameError: name 'read_delta' is not defined

In [185]:
df_2023 = df_receiving.filter((pl.col("Season")==2023) & pl.col("Player").str.starts_with("A")).limit(10)
display(df_2023)

Player,Rec,Yds,TD,20+,40+,LNG,Rec 1st,1st%,Rec FUM,Rec YAC/R,Tgts,Season
str,str,str,str,str,str,str,str,str,str,str,str,i32
"""Amon-Ra St. Br…","""119""","""1515""","""10""","""24""","""3""","""70""","""75""","""63""","""1""","""668""","""164""",2023
"""A.J. Brown""","""106""","""1456""","""7""","""21""","""3""","""59""","""70""","""66""","""2""","""473""","""158""",2023
"""Adam Thielen""","""103""","""1014""","""4""","""9""","""0""","""32""","""56""","""54.4""","""2""","""334""","""137""",2023
"""Alvin Kamara""","""75""","""466""","""1""","""3""","""0""","""25""","""23""","""30.7""","""0""","""501""","""86""",2023
"""Amari Cooper""","""72""","""1250""","""5""","""21""","""8""","""75""","""50""","""69.4""","""2""","""271""","""128""",2023
"""Austin Ekeler""","""51""","""436""","""1""","""5""","""0""","""39""","""21""","""41.2""","""2""","""533""","""74""",2023
"""Antonio Gibson…","""48""","""389""","""2""","""3""","""1""","""41""","""19""","""39.6""","""2""","""377""","""59""",2023
"""Allen Robinson…","""34""","""280""","""0""","""2""","""0""","""31""","""9""","""26.5""","""0""","""68""","""49""",2023
"""Alec Pierce""","""32""","""514""","""2""","""8""","""2""","""58""","""24""","""75""","""0""","""97""","""65""",2023


In [177]:
# Assuming all columns are strings (you might need to adjust data types based on your data)
df = df_receiving.select([
    pl.col("Player").cast(pl.Utf8),  # Cast Player to string (Utf8)
    pl.col("Rec").cast(pl.Int32),  # Cast Rec to integer (Int32)
    pl.col("Yds").cast(pl.Int32),  # Cast Yds to integer (Int32)
    pl.col("TD").cast(pl.Int32),   # Cast TD to integer (Int32)
    pl.col("Season").cast(pl.Utf8)
])

sorted_df = df.select("Player", "Rec", "Yds", "TD", "Season").sort("TD",descending=True)

display(sorted_df.head(10))

Player,Rec,Yds,TD,Season
str,i32,i32,i32,str
"""Randy Moss""",98,1493,23,"""2007"""
"""Jerry Rice""",65,1078,22,"""1987"""
"""Mark Clayton""",73,1389,18,"""1984"""
"""Sterling Sharp…",94,1119,18,"""1994"""
"""Davante Adams""",115,1374,18,"""2020"""
"""Jerry Rice""",82,1483,17,"""1989"""
"""Randy Moss""",111,1632,17,"""2003"""
"""Cris Carter""",122,1371,17,"""1995"""
"""Carl Pickens""",99,1234,17,"""1995"""


In [82]:
df = df_receiving.select([
    pl.col("Player").cast(pl.Utf8),  # Cast Player to string (Utf8)
    pl.col("Rec").cast(pl.Int32),  # Cast Rec to integer (Int32)
    pl.col("Yds").cast(pl.Int32),  # Cast Yds to integer (Int32)
    pl.col("TD").cast(pl.Int32),   # Cast TD to integer (Int32)
])

df_sum = df.group_by("Player",maintain_order=True).sum().sort("Yds",descending=True).head(15)
display(df_sum)

Player,Rec,Yds,TD
str,i32,i32,i32
"""Jerry Rice""",1549,22895,197
"""Steve Smith""",1407,18622,106
"""Larry Fitzgera…",1432,17492,121
"""Terrell Owens""",1078,15934,153
"""Randy Moss""",982,15292,156
"""Isaac Bruce""",1024,15208,91
"""Tony Gonzalez""",1325,15127,111
"""Tim Brown""",1094,14934,100
"""Marvin Harriso…",1102,14580,128
